In [4]:
!pip install -q sagemaker sagemaker[local]

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-kogpt2'

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='hunkimSagemaker')['Role']['Arn']
    
print(role)

arn:aws:iam::294038372338:role/hunkimSagemaker


In [3]:
%%bash
mkdir data
touch data/README.md

In [7]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-kogpt2


## Dummy train
It just store the models and voc data

In [5]:
from sagemaker.mxnet import MXNet

m = MXNet('train.py',
          source_dir='KoGPT2',
          role=role,
          train_instance_count=1,
          #train_instance_type='ml.c4.xlarge',
          train_instance_type='local',
          framework_version='1.6.0',
          py_version='py3',
          # distributions={'parameter_server': {'enabled': True}},
          # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,        
          hyperparameters={'batch_size': 8,
                           'epochs': 2,
                           'lr': 0.01})

In [6]:
m.fit(inputs)

NameError: name 'inputs' is not defined

In [ ]:
from sagemaker.mxnet.model import MXNetModel

training_job_name = m.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
print(trained_model_location)
#s3://sagemaker-us-west-2-294038372338/mxnet-training-2020-06-07-02-55-29-667/output/model.tar.gz

mm = MXNetModel(model_data=trained_model_location,
          role=role,
          framework_version='1.6.0',
          entry_point='infer.py',
          source_dir='KoGPT2',
          py_version='py3',
          model_server_workers=2)

In [ ]:
predictor = mm.deploy(initial_instance_count=1, 
                    instance_type='local'
                    #instance_type='ml.c5.xlarge')

In [ ]:
endpoint_name = predictor.endpoint
print(endpoint_name)